In [2]:
#@title Downloads
%%bash
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00000.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00001.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00002.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00003.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00004.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00005.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00006.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00007.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00008.json.gz"
wget "https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00009.json.gz"
mkdir data-engineer-test
mv part-*.gz data-engineer-test/
wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip -o .
pip install pyspark

Archive:  ngrok-stable-linux-amd64.zip
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=008c3c97e9e72bec5d08666b41ead39112ce0ca207ccd46703808511d08bac4e
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#@title Truncamento dos dados
# se estiver testando no jupyter notebook, rode esse script para truncar os dados
# é possível rodar os dados completos no jupyter, mas o job demora ≃~ 30 minutos
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    LongType,
    FloatType,
    StringType,
    StructType,
    StructField,
    IntegerType
)

a = os.listdir("data-engineer-test/")

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("EscaleTT") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .getOrCreate()

schema = StructType([
            StructField("n", IntegerType(), True),
            StructField("event", StringType(), True),
            StructField("version", FloatType(), True),
            StructField("platform", StringType(), True),
            StructField("os_family", StringType(), True),
            StructField("anonymous_id", StringType(), True),
            StructField("device_family", StringType(), True),
            StructField("browser_family", StringType(), True),
            StructField("device_sent_timestamp", StringType(), True)
    ]
)

for i in a:
    spark.read.schema(schema).json("data-engineer-test/"+i).limit(10000).write.json("data-engineer-test-trunc/"+i)

!ls data-engineer-test-trunc

part-00000.json.gz  part-00003.json.gz	part-00006.json.gz  part-00009.json.gz
part-00001.json.gz  part-00004.json.gz	part-00007.json.gz
part-00002.json.gz  part-00005.json.gz	part-00008.json.gz


In [4]:
#@title Monitoramento de jobs 
# tunelamento http para ver a spark-ui
# rode o código abaixo e use o link https://*.ngrok.io no arquivo de log para acessar o monitoramento
%%bash --bg
./ngrok http 4040 -log ngrok.log &


Starting job # 0 in a separate thread.


In [5]:
!cat ngrok.log

cat: ngrok.log: No such file or directory


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import (
    LongType,
    FloatType,
    StringType,
    StructType,
    StructField,
    IntegerType
)

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("EscaleTT") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .getOrCreate()

# use dados truncados para teste no jupyter notebook, use o .py para rodar um job no cluster
# é possível rodar os dados completos no jupyter, mas o job demora ≃~ 30 minutos 
datapath = "file:///content/data-engineer-test-trunc/part-00*.json.gz"
schema = StructType([
            StructField("n", IntegerType(), True),
            StructField("event", StringType(), True),
            StructField("version", FloatType(), True),
            StructField("platform", StringType(), True),
            StructField("os_family", StringType(), True),
            StructField("anonymous_id", StringType(), True),
            StructField("device_family", StringType(), True),
            StructField("browser_family", StringType(), True),
            StructField("device_sent_timestamp", StringType(), True)
    ]
)

df = spark \
    .read \
    .schema(schema) \
    .json(datapath) \
    .withColumn("file",f.input_file_name()) \
    .withColumn("device_sent_timestamp", f.substring(f.col("device_sent_timestamp"),0,10)) \
    .withColumn("device_sent_timestamp", f.from_unixtime(f.col("device_sent_timestamp")))


# FAZENDO SESSIONAMENTO
# IMPORTANTE: arredondei o tempo máximo de sessão pra 1 hora, ao invés de 30 minutos
df = df.withColumn("session_timestamp", f.date_trunc("hour","device_sent_timestamp")) 

session_cols = [
    # f.col("anonymous_id"), # IMPORTANTE:  anonymous_id gerava um hash único pra cada linha, decidi dropar
    # fingerprinting está sendo feito por User-Agent e timestamp
    f.col("os_family"),
    f.col("device_family"),
    f.col("browser_family"),
    f.col("session_timestamp") 
]
df = df.withColumn("session_id", f.sha2(f.concat(*session_cols),256))

# ETAPA 1
stage1 = df \
.groupBy(f.col("file")) \
.agg(f.countDistinct(f.col("session_id")).alias("unique_sessions")) \
.rdd.collectAsMap()

# ETAPA 2
def unique_sessions(family):
    return df \
    .groupBy(f.col(family)) \
    .agg(f.countDistinct(f.col("session_id")).alias("unique_sessions")) \
    .rdd.collectAsMap()

stage2 = {
    "os_family": unique_sessions("os_family"),
    "device_family": unique_sessions("device_family"),
    "browser_family": unique_sessions("browser_family")
} 

# ETAPA 3
session_duration = f.unix_timestamp(f.col("session_end")) - f.unix_timestamp(f.col("session_start"))
session_start_and_end = ( f.min("device_sent_timestamp").alias("session_start"), f.max("device_sent_timestamp").alias("session_end") )

df2 = df.groupBy(f.col("session_id")) \
    .agg(*session_start_and_end) \
    .withColumn("session_duration", session_duration)

df = df.join(df2,"session_id")

def median_session_duration(family):
    return df \
    .groupBy(f.col("os_family")) \
    .agg(f.expr('percentile_approx(session_duration, 0.5)').alias("median_duration")) \
    .rdd.collectAsMap()

stage3 = {
    "os_family": median_session_duration("os_family"),
    "browser_family": median_session_duration("browser_family"),
    "device_family": median_session_duration("device_family")
}

stage1,stage2,stage3

({'file:///content/data-engineer-test-trunc/part-00000.json.gz/part-00000-e6fdd502-aed2-4e31-81bd-84a24db8f5ee-c000.json': 713,
  'file:///content/data-engineer-test-trunc/part-00001.json.gz/part-00000-73e29a2a-cc82-4fb6-96c1-50dada2beefd-c000.json': 535,
  'file:///content/data-engineer-test-trunc/part-00002.json.gz/part-00000-3e83f63c-32ed-44a3-85a6-4a0bf5405a17-c000.json': 976,
  'file:///content/data-engineer-test-trunc/part-00003.json.gz/part-00000-a66dc1a3-9373-4b07-a718-3c0a84b9c25a-c000.json': 820,
  'file:///content/data-engineer-test-trunc/part-00004.json.gz/part-00000-0af48a5a-20ec-448f-bada-84883ac5cdf2-c000.json': 603,
  'file:///content/data-engineer-test-trunc/part-00005.json.gz/part-00000-deccbb3d-4f01-4644-ae69-d4662ffef7a4-c000.json': 764,
  'file:///content/data-engineer-test-trunc/part-00006.json.gz/part-00000-36180e68-74a2-413c-a91a-91267516b209-c000.json': 738,
  'file:///content/data-engineer-test-trunc/part-00007.json.gz/part-00000-4404bd36-3269-45e6-9ec0-04c776

In [7]:
import os
import json
import errno


def safe_write_json(dict_,path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
    with open(path,"w") as f:
        json.dump(dict_,f)


safe_write_json(stage1,"sessions/unique/by_file.json")
safe_write_json(stage1,"sessions/unique/by_family.json")
safe_write_json(stage1,"sessions/median/by_family.json")